# <center>**RADIOMICS EXTRACTION - Analyze and correct mask errors - OW**</center>

*Repetition of Step 4 for Ovak Window Segmentations Analysis of mask errors.*

(*Step 4*)

## **Radiomics Workflow:**
  
  **1. Download** DICOM images and convert to **NRRD.**

  **2.** Perform **target segmentations** and save in **NRRD.**
  
  **3.** Perform a **first Radiomic Feature** ***Dummy*** **Extraction,** to:
    
  - Detect **erros in segmentations**: only one dimension, no label 1, only one segmented voxel...
  - Analyze **bin width**.


  **4. Analyze** and **correct mask errors.**

  **5. Adjust bin width.**
   - **Tune featureextractor param file.**


  **6.** Perform **final Radiomic Feature Extraction.**

  **7. Clean Radiomic Features.**

  **8. Merge** with **labels** and **clinical data.**

  **9. Descriptive Statistics.**

  **10. Inferential Statistics.**

  **11. Machine Learning.**

## **MASK ANALYSIS - OW**

This notebook **checks and analyze** those **masks** which return **errors** during the first radiomic features extraction. The errors return by <code>pyradiomics</code> are the following.

**No label 1:**

* <code>Exception thrown in SimpleITK LabelShapeStatisticsImageFilter_GetBoundingBox: /tmp/SimpleITK-build/ITK-prefix/include/ITK-5.4/itkLabelMap.hxx:121:
ITK ERROR: LabelMap(0x561dda58da00): No label object with label 1.</code>

**Only one voxel:**

* <code>mask only contains 1 segmented voxel! Cannot extract features for a single voxel.</code>


To analyze this issues, the steps followed are:

0.   Environment **configuration**.
1.   Analyze **mask errors**.
      * No label 1.
      * Only one voxel.
2.   Check an error

In [1]:
# Install libraries
#pip install -q SimpleITK pyradiomics

In [2]:
import os
import numpy as np
import SimpleITK as sitk
from radiomics import featureextractor
import scipy # for filters

### **0. Environment configuration**

#### Set the working directory

In [3]:
# Set working directory
wd = '/Users/pablomenendezfernandez-miranda/Proyecto Otosclerosis/'
os.chdir(wd)

print(f'Directorio actual: {os.getcwd()}')

# Check directory files
print(f'Directory files: {os.listdir(wd)}')

Directorio actual: /Users/pablomenendezfernandez-miranda/Proyecto Otosclerosis
Directory files: ['Tables', 'Databases', '.DS_Store', 'Episodes', 'Figures', 'Notebooks']


#### Mount Drive (if executed on Google Colab)

In [4]:
#from google.colab import drive
#drive.mount('/content/drive')

### **1. Analyze mask errors**

In [5]:
dir= 'Episodes/'

#### **1.1. No label 1**

In [6]:
# No label 1
segmentations= ['EPI_0034_OW.nrrd']

for segmentation in segmentations:
  segmentation_dir =os.path.join(dir, segmentation)
  segmentation_itk = sitk.ReadImage(segmentation_dir)
  segmentation_array = sitk.GetArrayFromImage(segmentation_itk)
  print(f'Valores únicos de la segmentación {segmentation}: {np.unique(segmentation_array)}')

Valores únicos de la segmentación EPI_0034_OW.nrrd: [0 1]


#### **1.2. Only one voxel**

In [7]:
# Solo un vóxel segmentado
segmentations= ['EPI_0001_OW.nrrd', 'EPI_0004_OW.nrrd', 'EPI_0008_OW.nrrd', 'EPI_0028_OW.nrrd', 'EPI_0052_OW.nrrd', 'EPI_0063_OW.nrrd']

for segmentation in segmentations:
  segmentation_dir =os.path.join(dir, segmentation)
  segmentation_itk = sitk.ReadImage(segmentation_dir)
  segmentation_array = sitk.GetArrayFromImage(segmentation_itk)
  print(f'Valores etiquetados como 1 de la segmentación {segmentation}: {np.sum(segmentation_array == 1)}  (etiquetados como 0 {np.sum(segmentation_array == 0)})')

Valores etiquetados como 1 de la segmentación EPI_0001_OW.nrrd: 159  (etiquetados como 0 74973025)
Valores etiquetados como 1 de la segmentación EPI_0004_OW.nrrd: 166  (etiquetados como 0 50069338)
Valores etiquetados como 1 de la segmentación EPI_0008_OW.nrrd: 526  (etiquetados como 0 51379698)
Valores etiquetados como 1 de la segmentación EPI_0028_OW.nrrd: 191  (etiquetados como 0 65011521)
Valores etiquetados como 1 de la segmentación EPI_0052_OW.nrrd: 206  (etiquetados como 0 72875826)
Valores etiquetados como 1 de la segmentación EPI_0063_OW.nrrd: 223  (etiquetados como 0 73137953)


### **2. Check an error**

In [8]:
image= 'EPI_0001.nrrd'
segmentation= 'EPI_0001_OW.nrrd'

params= 'Notebooks/param_files/param_file_STAPES_tuned.yaml'
extractor= featureextractor.RadiomicsFeatureExtractor(params, verbose=True)

image_path= os.path.join(dir, image)
segmentation_path= os.path.join(dir, segmentation)

extractor.execute(image_path, segmentation_path, voxelBased=False)

ValueError: mask only contains 1 segmented voxel! Cannot extract features for a single voxel.